# 1 Metodi ereditati



JpaRepository in Spring Data JPA eredita diversi metodi da diverse interfacce, tra cui PagingAndSortingRepository e CrudRepository. Questi metodi forniscono una vasta gamma di funzionalità per l'accesso e la manipolazione dei dati in un database relazionale. Ecco alcuni dei metodi più comuni e utili ereditati da JpaRepository:

### 1) CRUD (Create, Read, Update, Delete) Operations:

- save(S entity): Salva un'entità nel database. Può essere utilizzato per inserire o aggiornare un'entità.
- saveAll(IterableSet&lt;S&gt; entities): Salva tutte le entità fornite nell'iterabile.
- findById(ID id): Recupera un'entità per il suo ID.
- findAll(): Restituisce tutte le entità.
- findAll(Sort sort): Restituisce tutte le entità ordinate.
- findAllById(IterableSet&lt;ID&gt; ids): Restituisce tutte le entità per gli ID forniti.
- deleteById(ID id): Elimina l'entità con l'ID specificato.
- delete(S entity): Elimina un'entità.
- deleteAll(Iterable&lt;? extends T&gt; entities): Elimina tutte le entità nell'iterabile fornito.
- deleteAll(): Elimina tutte le entità.

### 2) Paging and Sorting:

- findAll(Pageable pageable): Restituisce una Page di entità secondo l'oggetto Pageable fornito, che può includere informazioni di paginazione e ordinamento.
- findAll(Sort sort): Restituisce tutte le entità ordinate secondo l'oggetto Sort fornito.

### 3) Query Methods:

- count(): Restituisce il conteggio delle entità.
- existsById(ID id): Restituisce true se un'entità con l'ID specificato esiste, altrimenti false.

# 2 Metodi personalizati

Jpa creerà automaticamente l'implementazione dei metodi sulla base della sintassi con cui sono scritti. Possono ritornare solo entità o collezioni di entità.

La sintassi dei metodi personalizzati prevede tre parti differenti:

1) <b>Prefisso</b>
Indica l'azione. 
più usati:
    - find...By
    - read...By
    - query...By
    - get...By
    - count...By -> Long
    - exists...By -> boolean
    - delete...By
    - remove...By
2) <b>Soggetto</b>
Entità su cui si vuole effettuare la query. Deve avere lo stesso esatto nome della classe che la implementa, case sensitive.
3) <b>Condizione</b>
Condizione per ridurre l'ampiezza dei risultati. Possono essere combinate con And e Or.
Alcune parole chiave:
    - GreaterThan
    - LessThan
    - Containing
    - Like

# 3 Esempi JPQL

```java
    @Query("SELECT new it.cgmconsulting.myblog.dto.response.CommentResponse(" +
            "c.id, " +
            //"c.comment, " +
            "CASE WHEN (c.censored = true) THEN '*********' ELSE c.comment END, " +
            "c.user.username, " +
            "c.createdAt, " +
            "COALESCE(c.parent.id, 0) " +
            ") FROM Comment c " +
            "WHERE c.post.id = :postId " +
            "AND (c.post.publicationDate IS NOT NULL AND c.post.publicationDate < :now) " +
            "ORDER BY c.createdAt")
    List<CommentResponse> getCommentsByPostIdAndPublished(@Param("postId")int postId, LocalDateTime now);
```
Tramite JPQL possiamo restituire un oggetto (o una collection di oggetti) creato appositamente per i nostri scopi. In questo caso l'oggetto viene inizializzato nella SELECT tramitre new e tutto il path dei package in cui l'oggetto è inserito.<br>
I campi ritornati dovranno essere coerenti con il costruttore dell'oggetto.<br>
CASE WHEN è un costrutto simile all'if. In questo caso lo usiamo per estrarrre il valore dal campo c.censored e verificare il suo contenuto per definire l'oggetto da passare al costruttore di CommentResponse.<br>
COALESCE è un operatore che ritorna un valore di default indicato come secondo parametro dell'espressione nel caso in cui la query sia null su quel parametro.<br>
Le variabili del metodo possono essere usate nella query se precedute da :. Devono avere lo stesso nome del parametro o un nome indicato nell'annotazione @Param("nome") se diverso.

```java
    @Query(value="SELECT new it.cgmconsulting.myblog.dto.response.PostKeywordResponse(" +
            "p.id, " +
            "p.postTitle," +
            "p.publicationDate," +
            "p.user.username " +
            ") FROM Post p " +
            "WHERE p.publicationDate IS NOT NULL AND p.publicationDate < :now " +
            "ORDER BY p.publicationDate DESC")
    Page<PostKeywordResponse> getPaginatedPublishedPosts(@Param("now") LocalDateTime now, Pageable pageable);
```

Se voglio ritornare una risposta paginata, il valore di ritorno della query dovrà essere di tipo Page e il metodo dovrà prendere un parametro di tipo Pageable.

```java
    @Query(value="SELECT tag.id " +
    "FROM Post p " +
    "INNER JOIN p.tags tag " +
    "WHERE p.id = :postId AND tag.visible = true")
    Set<String> getTagsByPost(@Param("postId") long postId);


public class Post {
    .....
    @ManyToMany(fetch = FetchType.EAGER)
    @JoinTable(name = "post_tags",
            joinColumns = {@JoinColumn(name="post_id", referencedColumnName = "id")},
            inverseJoinColumns = {@JoinColumn(name="tag_id", referencedColumnName = "id")}
    )
    private Set<Tag> tags = new HashSet<>();
```

E' possibile utilizzare L'INNER JOIN al quale diamo un alias. JPQL lavora con le classi e non con le tabelle. Quindi per ogni classe definita nella query abbiamo la possibilità di accedere ai suoi campi. Per effettuare il JOIN è necessario che la classe che effettua il JOIN Post in questo caso abbia un riferimento alla classe su cui va ad effettuare il JOIN.<br>
In questo caso la relazione è @ManyToMany, il che vuol dire che Hibernate eseguirà un Lazy fetch di default. Quindi nel caso io voglia recuperare un oggetto tutte le Entità in una relazione...ToMany non verranno recuperate a meno che non si cambi il tipo di Fetch.

```java
    @Query(value="SELECT new it.cgmconsulting.myblog.dto.response.PostBoxResponse(" +
            "si.section.post.id as postId, " +
            "si.section.post.postTitle as postTitle, " +
            "si.section.sectionContent as sectionContent," +
            ":path || si.id || '.' || si.fileType as image, " +
            "(SELECT COALESCE(ROUND(AVG(r.rate),1), 0.0) FROM Rating r WHERE r.ratingId.post.id = si.section.post.id) as average, " +
            "si.section.post.publicationDate as publicationDate " +
            ") FROM SectionImages si " +
            "WHERE si.section.sectionType = :sectionType " +
            "AND (si.section.post.publicationDate IS NOT NULL AND si.section.post.publicationDate < :now) ")
    Page<PostBoxResponse> getBoxes(Pageable pageable, @Param("now") LocalDateTime now, SectionType sectionType, String path);
```

In questa query notiamo come è possibile acccedere a tutti i cmapi degli oggetti collegati nelle Entità. Quindi partendo da un oggetto posso recuperare dati anche di altre entità utilizzando la sintassi di java. Spring provvede alla creazione in background dei relativi JOIN per recuperare gli attributi richiesti.<br>
Quando creiamo un oggetto nella query possiamo anche usare l'operatore || per ottenere una concatenazione di stringhe.<br>
E' possibile inserire delle subquery.<br>
E' possibile attribuire alias anche agli attributi ritornati all'interno dell'oggetto. Questo è partiolarmente importante se si applica la paginazione. Infatti l'ordinamento della paginazione viene fatto tramite la scelta di un attributo usando il suo nome. Il nome dell'attributo dell'oggetto PAgeable deve rispecchiare il nome dell'attributo nell'oggetto

# 3 Esempi Query Derivate

```java
boolean existsBySectionTypeAndPost(SectionType sectionType, Post post);<br>
```
```java
List<String> getSectionsSectionContentByPostId(int postId);<br>
```
Anche nelle query derivte si fa riferimento alle entità e non alle tabelle. Le entità che vengono ritornate posso essere scritte al singolare e al plurale indipendentemente se ritorniamo o meno una collection. Il valore di ritorno della query determina la costruzione della sintassi SQL. E' possibile accedere ai campi dell'oggetto scrivendo i campi dell'oggetto dopo il nome di esso. In questo caso SectionSectionContent, PostId. da notare che Section ha un attributo di tipo Post relazionato all'entità Post, per cui possiamo accedere al campo di Post.id.

```java
Set<Tag> findAllByVisibleTrueOrderById();
```
In questo caso non specifichiamo il tipo di ritorno in quanto è estrapolato dalla Classe del repository stesoo in base ai parametri passati a JpaRepository. E' possibile specificare nella query il valore del parametro (ByVisibleTrue) e l'ordinamento in base all'attributo.

```java
Set<Tag> findByVisibleTrueAndIdIn(Set<String> tags);
```
In questo caso IdIn fa riferimento al parametro passato al metodo. I parametri vengono interpretati in base alla query. 

```java
Optional<User> findByEmail(String email);
```
E' possibile fare ritornare il wrapper Optional per gestire i valori null

```java
Optional<Post> findByIdAndPublicationDateIsNotNullAndPublicationDateBefore(int id, LocalDateTime now);
List<PostResponse> findAllByPublicationDateIsNotNullAndPublicationDateBefore(LocalDateTime now);
```

```java
boolean existsByEmailOrUsername(String email, String username);
boolean existsByEmailAndIdNot(String email, int id);
boolean existsByEmail(String email);
```
